In [ ]:
# قراءه من ملف csv 

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pocessing data ").getOrCreate()

df= spark.read.csv(
    "data.csv",
    header=True,
    inferSchema=True
)
df.show(5)
df.printSchema()

In [ ]:
# فهم الداتا مهم قبل اي عمليه كلين عليها 

# هنشوف الاعمده
df.columns

#عدد الصفوف 
df.count

#القيم  الفاضيه 

from pyspark.sql.functions import col 

df.select([
    col(c).isNull().alias(c) for c in df.columns]
).show(5)


In [ ]:
#حذف اعمده انا مش محتجها 
df = df.drop("unnecessary_coulmn","temp_col")
#  او ممكن نعمل سليكت لاعمده الي محتجنها بس 
df= df.select("id","name","age","date")

# =============================================
# حذف الصفوف الي فيها قيم فاضيه 

#حذف اي صف فيه null
df_clean= df.dropna()

#حذف صفوف علي اعمده معينه بس يعني احذف الصف لو  عند العمود الفلاني ملهوش فيمه او قيمه بنل 
df_clean = df.dropna(subset=["id","date"])



In [ ]:
#  ملي القيم الفاضيه  بطرق مختلفه 

# ملي القيم كلها بقيمه واحده 
df=df.fillna(0)

# ملي حسب العمود
df=df.fillna({
    "amount":0,
    "name":"unknown"
})

# ملي علي حسب المتوسط(mean)
from pyspark.sql.functions import avg

mean_value= df.select(avg("amount")).first()[0]
df=df.fillna({
    "amount":mean_value
})

In [ ]:
# تغير نوع الدتا (data type casting)
from pyspark.sql.functions import col

df=df.withColumn(
    'amount',
    col("amount").cast("double")
)

# تحويل تاريخ
from pyspark.sql.functions import to_date

df=df.withColumn(
    "order_date",
    to_date(col("order_date")),
    "yyy-MM-dd"
)

In [ ]:
# فلتره الداتا (filter/scope)

# فلتره شرطيه 
df=df.filter(col("amount")>1000)

# فلتره بتاريخ 
df =df.filter(
    col("order_date")>= "1-1-2024"
)

# الفلتره بتاريخين محددين او بين فتره معينه 
df=df.filter(
    (col("order_date")>= "1-1-2024")&
    (col("order_date")<= "30-6-2024")
)

In [ ]:
# انشاء اعمده جديده 
from pyspark.sql.functions import when

df= df.withColumn(
    "amount_category" ,# اسم العمود 
    when(col("amount") >= 1000,"high") # القيم الي هتبقا في العمود علي حسب الشرط بتاعنا 
    .when(col("amount")>=500, "medium")
    .otherwise("low")

)

In [ ]:
#ازاله القيم المكرره 
df= df.dropDuplicates()

# ازاله القيم المكرره علي حسب عمود معين 
df=df.drop_duplicates(["id"])

In [ ]:
#agregations 

df.groupBy("amount_category").count().show(10)